In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import copy


import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as ds
from torchvision import models, transforms, utils, datasets
from torch.utils import data
from torchsummary import summary
from torch.optim import lr_scheduler

# Data Read-In and Loading

In [2]:
fish_df = pd.read_csv("data/fish_df.csv")

In [3]:
is_fish_df = pd.read_csv("data/is_fish.csv").iloc[:,1:]
is_fish_df["local_paths"] = is_fish_df["Species"].astype(str) + "/" + is_fish_df["Filename"]
path_set = set(is_fish_df["local_paths"])

In [4]:
species_count = 92 #len(is_fish_df["Species"].unique())

In [5]:
# We normalize to imagenet mean for the data (https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Resize([224, 224])]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Resize([224, 224])]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Resize([224, 224])])}

In [6]:
data_dirs = ["/home/shivaram/DS/Projects/FishID/data/model_data/is_fish_scientific/", "/home/shivaram/DS/Projects/FishID/data/model_data/is_fish_common/", "/home/shivaram/DS/Projects/FishID/data/model_data/is_fish_mixed/"]
diff_image_datasets = {dd:{x: datasets.ImageFolder(os.path.join(dd, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']} for dd in data_dirs}

In [7]:
image_datasets = diff_image_datasets["/home/shivaram/DS/Projects/FishID/data/model_data/is_fish_scientific/"]

In [8]:
batch_size = 32#64
#epoch_samples = 2560# len(samples_weight)

In [9]:
weighted_samplers = {}
for subset in ["train", "val", "test"]:
    target = image_datasets[subset].targets
    
    if True:#subset == "train":
        class_sample_count =np.array([ len(np.where(target == t)[0]) for t in np.unique(target)])
        weight = 1. / class_sample_count
        samples_weight = np.array([weight[t] for t in target])
        samples_weight = torch.from_numpy(samples_weight)
        epoch_samples = len(samples_weight)
        sampler = data.WeightedRandomSampler(samples_weight, epoch_samples)
        weighted_samplers[subset] = sampler
    else:
        sampler = data.RandomSampler(image_datasets[subset])
        weighted_samplers[subset] = sampler
    


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], sampler = weighted_samplers[x], 
                                              batch_size=batch_size, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

# Model Training and Evaluation Methods

In [10]:
import ModelMethods

In [11]:
def train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=25, verbose = True):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        if verbose:
            print(f'Epoch {epoch + 1}/{num_epochs}')
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if verbose:
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    if verbose:
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc


In [12]:
def test_model(model, criterion, phase = "test", verbose = True):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]

    if verbose:
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
    return epoch_acc

In [13]:
def train_test_save_model(save_path, dataloaders, model, criterion, optimizer, scheduler, num_epochs = 25, verbose = True):
    
    trained_model, val_acc = train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs, verbose) 
    torch.save(model.state_dict(), save_path)
    test_acc = test_model(trained_model, criterion)
    
    if verbose:
        print(f"Val Accuracy: {val_acc}")
        print(f"Test Accuracy: {test_acc}")
        

    return trained_model, val_acc, test_acc

# Determining the Best Model

In [14]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3060 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## ResNet18 

In [15]:
resnet18_path = "models/92_classifier/resnet18.pt"

model_ft = models.resnet18(pretrained=True)
#for param in model_ft.parameters():
#    param.requires_grad = False
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, species_count)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor = .2, patience = 3)


if os.path.exists(resnet18_path):
    model_ft.load_state_dict(torch.load(resnet18_path))
    test_model(model_ft, criterion)
else:
    model_ft, val_acc, test_acc = train_test_save_model(resnet18path, dataloaders, model_ft, criterion, optimizer_ft, exp_lr_scheduler, 30, True)

test Loss: 2.9258 Acc: 0.3972


### Efficient Net B0

In [15]:
efficientnetb0_path = "models/92_classifier/efficient_netb0.pt"

model_ft =  models.efficientnet_b0(pretrained = True)
#for param in model_ft.parameters():
#    param.requires_grad = False
num_ftrs = model_ft.classifier[1].in_features
model_ft.fc = nn.Linear(num_ftrs, species_count)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor = .2, patience = 3)

if os.path.exists(efficientnetb0_path):
    model_ft.load_state_dict(torch.load(efficientnetb0_path))
    test_model(model_ft, criterion)
else:
    model_ft, val_acc, test_acc = train_test_save_model(efficientnetb0_path, dataloaders, model_ft, criterion, optimizer_ft, exp_lr_scheduler, 30, True)

RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 7.79 GiB total capacity; 4.91 GiB already allocated; 49.31 MiB free; 5.04 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:

print(test_acc)

Epoch 1/30
----------
train Loss: 3.6540 Acc: 0.1590
val Loss: 3.3321 Acc: 0.2377

Epoch 2/30
----------
train Loss: 2.5818 Acc: 0.3563
val Loss: 2.3511 Acc: 0.4305

Epoch 3/30
----------
train Loss: 2.1024 Acc: 0.4702
val Loss: 2.2420 Acc: 0.4532

Epoch 4/30
----------
train Loss: 1.8177 Acc: 0.5344
val Loss: 2.0545 Acc: 0.5145

Epoch 5/30
----------
train Loss: 1.6082 Acc: 0.5854
val Loss: 1.7844 Acc: 0.5485

Epoch 6/30
----------
train Loss: 1.4358 Acc: 0.6278
val Loss: 1.7463 Acc: 0.5508

Epoch 7/30
----------
train Loss: 1.3037 Acc: 0.6576
val Loss: 1.8550 Acc: 0.5718

Epoch 8/30
----------
train Loss: 0.9562 Acc: 0.7487
val Loss: 1.5306 Acc: 0.6597

Epoch 9/30
----------
train Loss: 0.8376 Acc: 0.7783
val Loss: 1.5031 Acc: 0.6591

Epoch 10/30
----------
train Loss: 0.7444 Acc: 0.8006
val Loss: 1.5208 Acc: 0.6693

Epoch 11/30
----------
train Loss: 0.6593 Acc: 0.8223
val Loss: 1.4498 Acc: 0.6858

Epoch 12/30
----------
train Loss: 0.6072 Acc: 0.8358
val Loss: 1.5737 Acc: 0.6784

E

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 7.79 GiB total capacity; 4.54 GiB already allocated; 46.31 MiB free; 4.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Todo
- Train Standard Network
- Adjust sample weighting
- Train Siamese Network or GAN
- Refactor code
- [Don't weighted random sample for validation and test](https://discuss.pytorch.org/t/dataloader-using-subsetrandomsampler-and-weightedrandomsampler-at-the-same-time/29907/7)